In [112]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sklearn
from scipy.linalg import null_space

In [113]:
#Generate Data
iris = sklearn.datasets.load_iris()
iris_data = iris.data - iris.data.mean(axis=0)
iris_data.shape

(150, 4)

In [141]:
# we have 4 variables with each having 150 observations 
# the above will give a k by k, sample covareince matrix 
# from scartch and barely vectorized not optimized nor libraries used
def Covariance_m(matrix):
    size = matrix.shape[1]
    Sample_covariance_m = np.zeros((size, size))
    means = np.mean(matrix, axis=0)
    centered_data = matrix - means
    for j in range(len(Sample_covariance_m)):
        for k in range(len(Sample_covariance_m)):
            Sample_covariance_m[j , k] = np.sum(centered_data[:,j] * centered_data[:,k]) / (matrix.shape[0] -1)
    return Sample_covariance_m
def proj(b, a):
    mul = ((np.dot(a, b)) / (np.dot(b, b)))
    return mul * b
def QR_factor(matrix):
    size = len(matrix)
    Q, R , u = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))
    for k in range(size):
        if k == 0:
            u[k] = matrix[:,0]
        else:
            final_sum = 0
            for j in range(k):
                sum = proj(u[j], matrix[:,k])
                final_sum += sum
            u[k] = matrix[:, k] - final_sum
    for index in range(size):
        e_i = u[index] / np.linalg.norm(u[index])
        Q[:,index] = e_i
    for i in range(size):
        for j in range(i , size):
            R[i, j] = np.dot(Q[:,i], matrix[:,j])
    return Q , R
def QR_alg(A_0):
    Q, R = QR_factor(A_0)
    for _ in range(10000):
        A_0 = np.dot(R, Q)
        Q, R = QR_factor(A_0)
    return A_0
def eigenvalues(matrix):
    eigen = QR_alg(matrix)
    eigenvalues = []
    for i in range(len(eigen)):
        for j in range(i, len(eigen)):
            if i == j:
                eigenvalues.append(eigen[i, j])
    return eigenvalues

def eigenvectors(matrix):
    size = len(matrix)
    eigenvectors = np.zeros((size, size))
    eigenvalues_list = eigenvalues(matrix)
    eigenvalues_list = sorted(eigenvalues_list)

    for index, eigenvalue in enumerate(eigenvalues_list):

        A = matrix - eigenvalue * np.identity(size, dtype=None, like=None)
        eigenvector = null_space(A)
        eigenvectors[:, index] = eigenvector.T / np.linalg.norm(eigenvector)
    return eigenvectors




        

    


In [142]:
s = Covariance_m(iris_data)
Q, R = QR_factor(s)


In [143]:
k = eigenvalues(s)

In [144]:
eigenvectors(s)

array([[ 0.31548719, -0.58202985, -0.65658877,  0.36138659],
       [-0.3197231 ,  0.59791083, -0.73016143, -0.08452251],
       [-0.47983899,  0.07623608,  0.17337266,  0.85667061],
       [ 0.75365743,  0.54583143,  0.07548102,  0.3582892 ]])

In [145]:
np.linalg.eig(s)

EigResult(eigenvalues=array([4.22824171, 0.24267075, 0.0782095 , 0.02383509]), eigenvectors=array([[ 0.36138659, -0.65658877, -0.58202985,  0.31548719],
       [-0.08452251, -0.73016143,  0.59791083, -0.3197231 ],
       [ 0.85667061,  0.17337266,  0.07623608, -0.47983899],
       [ 0.3582892 ,  0.07548102,  0.54583143,  0.75365743]]))